In [35]:
import imaplib
import email
from email.header import decode_header

In [37]:
EMAIL = "jaideep2022singh@gmail.com"
APP_PASSWORD = "ygklbmlefocmffim"

In [39]:
imap = imaplib.IMAP4_SSL("imap.gmail.com",993)
imap.login(EMAIL, APP_PASSWORD)

('OK', [b'jaideep2022singh@gmail.com authenticated (Success)'])

In [41]:
imap.select("inbox")

('OK', [b'5676'])

In [43]:
status, messages = imap.search(None, 'UNSEEN')  # Fetch unread only
email_ids = messages[0].split()
#print(status)

In [45]:
email_ids = email_ids[-5:]
emails = []

In [47]:
for eid in email_ids:
    _, msg_data = imap.fetch(eid, "(RFC822)")
    raw_email = email.message_from_bytes(msg_data[0][1])

    # Decode subject
    subject, encoding = decode_header(raw_email["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding or "utf-8", errors="ignore")

    # Get other headers
    sender = raw_email.get("From")
    date = raw_email.get("Date")

    # Extract plain text body
    body = ""
    if raw_email.is_multipart():
        for part in raw_email.walk():
            if part.get_content_type() == "text/plain" and "attachment" not in str(part.get("Content-Disposition")):
                try:
                    body = part.get_payload(decode=True).decode("utf-8", errors="ignore")
                except:
                    body = "Unable to decode body."
                break
    else:
        body = raw_email.get_payload(decode=True).decode("utf-8", errors="ignore")

    emails.append({
        "From": sender,
        "Subject": subject,
        "Date": date,
        "Body": body
    })

imap.logout()

('BYE', [b'LOGOUT Requested'])

In [49]:
for i, mail in enumerate(emails, start=1):
    print(f"\n📧 Email #{i}")
    print(f"From   : {mail['From']}")
    print(f"Date   : {mail['Date']}")
    print(f"Subject: {mail['Subject']}")
    print(f"Body   : {mail['Body'][:300]}...") 


📧 Email #1
From   : Tanish Khera <invitations@linkedin.com>
Date   : Tue, 15 Jul 2025 20:18:38 +0000 (UTC)
Subject: I want to connect
Body   : Tanish is waiting for your response
        
Hi Jaideep, I’d like to join your professional network

Tanish Khera
Aspiring Cybersecurity Analyst | AI & ML Entusiast
Jalandhar
27 connections in common
See all connections in common:https://www.linkedin.com/comm/search/results/people/?facetNetw...

📧 Email #2
From   : Twine Team <hello@twinehq.com>
Date   : Tue, 15 Jul 2025 22:33:38 +0000
Subject: 💼
Body   : <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional //EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="viewport" content="width=device-wi...

📧 Email #3
From   : Learning Lifecycle Management System <L2MS@sap.com>
Date   : Wed, 16 Jul 2025 03:30:09 +0000
Subject: SAP Learning Hub, studen

In [51]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [52]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaideepsingh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jaideepsingh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [59]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [61]:

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)  # remove URLs
    text = re.sub(r'[^a-z\s]', '', text)        # remove special chars & digits
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [63]:
for email in emails:
    full_text = email['Subject'] + ' ' + email['Body']
    email['Cleaned_Text'] = clean_text(full_text)

In [65]:
import joblib

In [67]:
model = joblib.load("spam_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")     # Trained TF-IDF vectorizer

In [73]:
for email in emails:
    vector = vectorizer.transform([email['Cleaned_Text']])
    prediction = model.predict(vector)[0]
    prob = model.predict_proba(vector)[0][1]  # Confidence of being spam

    email['Prediction'] = 'Spam' if prediction == 1 else 'Not Spam'
    email['Confidence'] = round(prob * 100, 2)



In [17]:
import pandas as pd

df = pd.DataFrame(emails)
df = df[['From', 'Date', 'Subject', 'Prediction', 'Confidence', 'Cleaned_Text']]  # Optional filter
df.to_csv("classified_emails.csv", index=False)
